In [32]:
import os
import networkx as nx
import numpy as np
import re
import glob

####### Global options #######
import configuration

pi = np.pi

vname_dict = {'V':1,'Er':2,'Ti':3,'Ce':4,'S':5,
			  'H':6,'He':7,'Li':8,'Be':9,'B':10,
			  'C':11,'N':12,'O':13,'F':14,'Ne':15,
			  'Na':16,'Mg':17,'Al':18,'Si':19,'P':20 ,
			  'Cl':21,'Ar':22,'K':23,'Ca':24,'Sc':24,
			  'Cr':26,'Mn':27,'Fe':28,'Co':29,'Ni':30}

metal_elements = ['Ac','Ag','Al','Am','Au','Ba','Be','Bi',
				  'Bk','Ca','Cd','Ce','Cf','Cm','Co','Cr',
				  'Cs','Cu','Dy','Er','Es','Eu','Fe','Fm',
				  'Ga','Gd','Hf','Hg','Ho','In','Ir',
				  'K','La','Li','Lr','Lu','Md','Mg','Mn',
				  'Mo','Na','Nb','Nd','Ni','No','Np','Os',
				  'Pa','Pb','Pd','Pm','Pr','Pt','Pu','Ra',
				  'Rb','Re','Rh','Ru','Sc','Sm','Sn','Sr',
				  'Ta','Tb','Tc','Th','Ti','Tl','Tm','U',
				  'V','W','Y','Yb','Zn','Zr']


####### Global options #######
IGNORE_ALL_ERRORS = configuration.IGNORE_ALL_ERRORS
#PRINT = configuration.PRINT
PRINT =True
CONNECTION_SITE_BOND_LENGTH = configuration.CONNECTION_SITE_BOND_LENGTH
WRITE_CHECK_FILES = configuration.WRITE_CHECK_FILES
WRITE_CIF = configuration.WRITE_CIF
ALL_NODE_COMBINATIONS = configuration.ALL_NODE_COMBINATIONS
USER_SPECIFIED_NODE_ASSIGNMENT = configuration.USER_SPECIFIED_NODE_ASSIGNMENT
COMBINATORIAL_EDGE_ASSIGNMENT = configuration.COMBINATORIAL_EDGE_ASSIGNMENT
#CHARGES = configuration.CHARGES
CHARGES = False
SCALING_ITERATIONS = configuration.SCALING_ITERATIONS
SYMMETRY_TOL = configuration.SYMMETRY_TOL
BOND_TOL = configuration.BOND_TOL
ORIENTATION_DEPENDENT_NODES = configuration.ORIENTATION_DEPENDENT_NODES
PLACE_EDGES_BETWEEN_CONNECTION_POINTS = configuration.PLACE_EDGES_BETWEEN_CONNECTION_POINTS
RECORD_CALLBACK = configuration.RECORD_CALLBACK
OUTPUT_SCALING_DATA = configuration.OUTPUT_SCALING_DATA
FIX_UC = configuration.FIX_UC
MIN_CELL_LENGTH = configuration.MIN_CELL_LENGTH
OPT_METHOD = configuration.OPT_METHOD
PRE_SCALE = configuration.PRE_SCALE
SINGLE_METAL_MOFS_ONLY = configuration.SINGLE_METAL_MOFS_ONLY
MOFS_ONLY = configuration.MOFS_ONLY
MERGE_CATENATED_NETS = configuration.MERGE_CATENATED_NETS
RUN_PARALLEL = configuration.RUN_PARALLEL
REMOVE_DUMMY_ATOMS = configuration.REMOVE_DUMMY_ATOMS

from ciftemplate2graph import ct2g
from vertex_edge_assign import vertex_assign, assign_node_vecs2edges
from cycle_cocyle import cycle_cocyle, Bstar_alpha
from bbcif_properties import cncalc, bbelems
from SBU_geometry import SBU_coords
from scale import scale
from scaled_embedding2coords import omega2coords
from place_bbs import scaled_node_and_edge_vectors, place_nodes, place_edges
from remove_net_charge import fix_charges
from remove_dummy_atoms import remove_Fr
from adjust_edges import adjust_edges
from write_cifs import write_check_cif, write_cif, bond_connected_components, distance_search_bond, fix_bond_sym, merge_catenated_cifs
from scale_animation import scaling_callback_animation, write_scaling_callback_animation, animate_objective_minimization
import itertools
from random import choice

In [33]:
# ct2g(template.CIF)
# SG, start, unit_cell, set(cns), set(e_types), cifname, aL, bL, cL, alpha, beta, gamma, max_le, catenation

#@TG template graph
#@start fractional coordinate
#@unit_cell Cartesian coordinate
#@TVT Template Vertex Table(the degree (number of connections) of node n in the main graph G, node type)
#@TET template ((l[0],l[1]))) edge connected node pair (only numeric part of the node identifier)
#@TNAME template name
#@catenation if G has seperated (connected) subgraph

## import matplotlib.pyplot as pl
## G1 = nx.path_graph(4)
## nx.add_path(G1, [10, 11, 12])
## nx.add_path(G1, [9, 11, 13])
## l = [len(c) for c in sorted(nx.connected_components(G1), key=len, reverse=True)]
## print(l,sorted(nx.connected_components(G1)))
## pl.figure()
## nx.draw_networkx(G1)
## pl.show()

In [34]:
template = 'fcu.cif'
supercell= np.array([1,1,1])
box_bound= supercell+1
#template = 'fcu.cif' #UIO6x

template_dir = 'template_database'
#template = 'rna.cif'
#nodes_dir = 'nodesdatabase'
nodes_dir = 'nodes'
edges_dir = 'edges'


In [35]:

PLACE_EDGES_BETWEEN_CONNECTION_POINTS = True
PRINT=False
print()
print('=========================================================================================================')
print('template :',template)                                          
print('=========================================================================================================')
print()
	
cat_count = 0
for net in ct2g(template,template_dir):

		cat_count += 1
		TG, start, unit_cell, TVT, TET, TNAME, a, b, c, ang_alpha, ang_beta, ang_gamma, max_le, catenation = net

		TVT = sorted(TVT, key=lambda x:x[0], reverse=True) # sort node with connected degree, the first one is the highest(full)-coordinated node
		TET = sorted(TET, reverse=True) #sort node_pair by the node_index
		#get node cif information from node dir
		
		node_cns = [(cncalc(node, nodes_dir), node) for node in os.listdir(nodes_dir)]

		print('Number of vertices = ', len(TG.nodes()))
		print('Number of edges = ', len(TG.edges()))
		print()

		edge_counts = dict((data['type'],0) for e0,e1,data in TG.edges(data=True))
		for e0,e1,data in TG.edges(data=True):
			edge_counts[data['type']] += 1
		
		if PRINT:
	
			print('There are', len(TG.nodes()), 'vertices in the voltage graph:')
			print()
			v = 0
	
			for node in TG.nodes():
				v += 1
				print(v,':',node)
				node_dict = TG.nodes[node]
				print('type : ', node_dict['type'])
				print('cartesian coords : ', node_dict['ccoords'])
				print('fractional coords : ', node_dict['fcoords'])
				#print('degree : ', node_dict['cn'][0])
				print()
	
			print('There are', len(TG.edges()), 'edges in the voltage graph:')
			print()
	
			for edge in TG.edges(data=True,keys=True):
				edge_dict = edge[3]
				ind = edge[2]
				print(ind,':',edge[0],edge[1])
				print('length : ',edge_dict['length'])
				print('type : ',edge_dict['type'])
				print('label : ',edge_dict['label'])
				print('positive direction :',edge_dict['pd'])
				print('cartesian coords : ',edge_dict['ccoords'])
				print('fractional coords : ',edge_dict['fcoords'])
				print()
	
		vas = vertex_assign(nodes_dir,TG, TVT, node_cns, unit_cell, USER_SPECIFIED_NODE_ASSIGNMENT, SYMMETRY_TOL, ALL_NODE_COMBINATIONS)
		CB,CO = cycle_cocyle(TG)

		for va in vas:
			if len(va) == 0:
				print('At least one vertex does not have a building block with the correct number of connection sites.')
				print('Moving to the next template...')
				print()
				continue
	
		if len(CB) != (len(TG.edges()) - len(TG.nodes()) + 1):
			print('The cycle basis is incorrect.')
			print('The number of cycles in the cycle basis does not equal the rank of the cycle space.')
			print('Moving to the next template...')
			continue
		
		num_edges = len(TG.edges())
		Bstar, alpha = Bstar_alpha(CB,CO,TG,num_edges)

		if PRINT:
			print('B* (top) and alpha (bottom) for the barycentric embedding are:')
			print()
			for i in Bstar:
				print(i)
			print()
			for i in alpha:
				print(i)
			print()
	
		num_vertices = len(TG.nodes())
	
		if COMBINATORIAL_EDGE_ASSIGNMENT:
			eas = list(itertools.product([e for e in os.listdir('edges')], repeat = len(TET)))
		else:
			edge_files = sorted([e for e in os.listdir('edges')])
			eas = []
			i = 0
			while len(eas) < len(TET):
				eas.append(edge_files[i])
				i += 1
				if i == len(edge_files):
					i = 0
			eas = [eas]
	
		g = 0

		for va in vas:
			#check if assigned node has metal element 
			node_elems = [bbelems(i[1], nodes_dir) for i in va]
			metals = [[i for i in j if i in metal_elements] for j in node_elems]
			metals = list(set([i for j in metals for i in j]))
			#set node cif files as vertex assignment
			v_set0 = [('v' + str(vname_dict[re.sub('[0-9]','',i[0])]), i[1]) for i in va]
			v_set1 = sorted(list(set(v_set0)), key=lambda x: x[0])
			v_set = [v[0] + '-' + v[1] for v in v_set1]
	
			print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
			print('vertex assignment : ',v_set)
			print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
			print()

			if SINGLE_METAL_MOFS_ONLY and len(metals) != 1:
				print(v_set, 'contains no metals or multiple metal elements, no cif will be written')
				print()
				continue

			if MOFS_ONLY and len(metals) < 1:
				print(v_set, 'contains no metals, no cif will be written')
				print()
				continue
			
			# add cifname to TG.nodes
			for v in va:
				for n in TG.nodes(data=True):
					if v[0] == n[0]:
						n[1]['cifname'] = v[1]
			
			for ea in eas:
	
				g += 1
	
				print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
				print('edge assignment : ',ea)
				print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
				print()
				
				type_assign = dict((k,[]) for k in sorted(TET, reverse=True))
				for k,m in zip(TET,ea):
					type_assign[k] = m
				
				# add cifname to TG.edge
				for e in TG.edges(data=True):
					ty = e[2]['type']
					for k in type_assign:
						if ty == k or (ty[1],ty[0]) == k:
							e[2]['cifname'] = type_assign[k]

				num_possible_XX_bonds = 0
				for edge_type, cifname in zip(TET, ea):
					if cifname == 'ntn_edge.cif':
						factor = 1
					else:
						factor = 2
					edge_type_count = edge_counts[edge_type]
					num_possible_XX_bonds += factor * edge_type_count

				ea_dict = assign_node_vecs2edges(nodes_dir,TG, unit_cell, SYMMETRY_TOL, template)
				all_SBU_coords = SBU_coords(TG, ea_dict, CONNECTION_SITE_BOND_LENGTH)
				sc_a, sc_b, sc_c, sc_alpha, sc_beta, sc_gamma, sc_covar, Bstar_inv, max_length, callbackresults, ncra, ncca, scaling_data = scale(all_SBU_coords,a,b,c,ang_alpha,ang_beta,ang_gamma,max_le,num_vertices,Bstar,alpha,num_edges,FIX_UC,SCALING_ITERATIONS,PRE_SCALE,MIN_CELL_LENGTH,OPT_METHOD)
		
				print('*******************************************')
				print('The scaled unit cell parameters are : ')
				print('*******************************************')
				print('a    :', np.round(sc_a, 5))
				print('b    :', np.round(sc_b, 5))
				print('c    :', np.round(sc_c, 5))
				print('alpha:', np.round(sc_alpha, 5))
				print('beta :', np.round(sc_beta, 5))
				print('gamma:', np.round(sc_gamma, 5))
				print()
	
				for sc, name in zip((sc_a, sc_b, sc_c), ('a', 'b', 'c')):
					cflag = False
					if sc == MIN_CELL_LENGTH:
						print('unit cell parameter', name, 'may have collapsed during scaling!')
						print('try re-running with', name, 'fixed or a larger MIN_CELL_LENGTH')
						print('no cif will be written')
						cflag = True
	
				if cflag:
					continue
	
				scaled_params = [sc_a,sc_b,sc_c,sc_alpha,sc_beta,sc_gamma]
			
				sc_Alpha = np.r_[alpha[0:num_edges-num_vertices+1,:], sc_covar]
				sc_omega_plus = np.dot(Bstar_inv, sc_Alpha)
			
				ax = sc_a
				ay = 0.0
				az = 0.0
				bx = sc_b * np.cos(sc_gamma * pi/180.0)
				by = sc_b * np.sin(sc_gamma * pi/180.0)
				bz = 0.0
				cx = sc_c * np.cos(sc_beta * pi/180.0)
				cy = (sc_c * sc_b * np.cos(sc_alpha * pi/180.0) - bx * cx) / by
				cz = (sc_c ** 2.0 - cx ** 2.0 - cy ** 2.0) ** 0.5
				sc_unit_cell = np.asarray([[ax,ay,az],[bx,by,bz],[cx,cy,cz]]).T
				
				scaled_coords = omega2coords(start, TG, sc_omega_plus, (sc_a,sc_b,sc_c,sc_alpha,sc_beta,sc_gamma), num_vertices, template, g, WRITE_CHECK_FILES)
				nvecs,evecs,node_placed_edges = scaled_node_and_edge_vectors(scaled_coords, sc_omega_plus, sc_unit_cell, ea_dict)
				placed_nodes, node_bonds = place_nodes(nvecs, nodes_dir,CHARGES, ORIENTATION_DEPENDENT_NODES)
				placed_edges, edge_bonds = place_edges(evecs, edges_dir,CHARGES, len(placed_nodes))
				#print(f"place_edges{place_edges}")
	
				if RECORD_CALLBACK:
	
					vnames = '_'.join([v.split('.')[0] for v in v_set])
	
					if len(ea) <= 5:
						enames = '_'.join([e[0:-4] for e in ea])
					else:
						enames = str(len(ea)) + '_edges'
	
					prefix = template[0:-4] + '_' +  vnames + '_' + enames
	
					frames = scaling_callback_animation(callbackresults, alpha, Bstar_inv, ncra, ncca, num_vertices, num_edges, TG, template, g, False)
					write_scaling_callback_animation(frames, prefix)
					animate_objective_minimization(callbackresults, prefix)
	
				if PLACE_EDGES_BETWEEN_CONNECTION_POINTS:
					placed_edges,cleaved_placed_edges,cleaved_placed_nodes,X_Opair = adjust_edges(placed_edges, placed_nodes, sc_unit_cell)
				
				# add classifination 
				#cleaved_placed_edges = placed_OXedges
				cleaved_placed_nodes = np.c_[cleaved_placed_nodes, np.array(['NODE' for i in range(len(cleaved_placed_nodes))])]
				cleaved_placed_edges = np.c_[cleaved_placed_edges, np.array(['EDGE' for i in range(len(cleaved_placed_edges))])]
				placed_nodes = np.c_[placed_nodes, np.array(['node' for i in range(len(placed_nodes))])]
				placed_edges = np.c_[placed_edges, np.array(['edge' for i in range(len(placed_edges))])]
				

				placed_all = list(placed_nodes) + list(placed_edges)
				bonds_all = node_bonds + edge_bonds
		
				if WRITE_CHECK_FILES:
					write_check_cif(template, placed_nodes, placed_edges, g, scaled_params, sc_unit_cell)
			
				if REMOVE_DUMMY_ATOMS:
					placed_all, bonds_all, nconnections = remove_Fr(placed_all,bonds_all)
				
				print('computing X-X bonds...')
				print()
				print('*******************************************')
				print('Bond formation : ')
				print('*******************************************')
				
				#fixed_bonds, nbcount, bond_check_passed = bond_connected_components(placed_all, bonds_all, sc_unit_cell, max_length, BOND_TOL, nconnections, num_possible_XX_bonds)
				#print('there were ', nbcount, ' X-X bonds formed')
				bond_check_passed =False
				if bond_check_passed:
					print('bond check passed')
					bond_check_code = ''
				else:
					print('bond check failed, attempting distance search bonding...')
					fixed_bonds, nbcount = distance_search_bond(placed_all, bonds_all, sc_unit_cell, 2.5)
					bond_check_code = '_BOND_CHECK_FAILED'
					print('there were', nbcount, 'X-X bonds formed')
				print()
		
				if CHARGES:
					fc_placed_all, netcharge, onetcharge, rcb = fix_charges(placed_all)
				else:
					fc_placed_all = placed_all
			
				fc_placed_all = placed_all
				fixed_bonds = fix_bond_sym(fixed_bonds, placed_all, sc_unit_cell)
	
				if CHARGES:
					print('*******************************************')
					print('Charge information :                       ')
					print('*******************************************')
					print('old net charge                  :', np.round(onetcharge, 5))
					print('rescaling magnitude             :', np.round(rcb, 5))
			
					remove_net = choice(range(len(fc_placed_all)))
					fc_placed_all[remove_net][4] -= np.round(netcharge, 4)
			
					print('new net charge (after rescaling):', np.sum([li[4] for li in fc_placed_all]))
					print()

				vnames = '_'.join([v.split('.')[0] for v in v_set])
				enames_list = [e[0:-4] for e in ea]
				enames_grouped = [list(edge_gr) for ind,edge_gr in itertools.groupby(enames_list)]
				enames_grouped = [(len(edge_gr), list(set(edge_gr))) for edge_gr in enames_grouped]
				enames_flat = [str(L) + '-' + '_'.join(names) for L,names in enames_grouped]
				enames = '_'.join(enames_flat)
				
				if catenation:
					outcifname = template[0:-4] + '_' +  vnames + '_' + enames + bond_check_code + '_' + 'CAT' + str(cat_count) + '.cif'
				else:
					outcifname = template[0:-4] + '_' +  vnames + '_' + enames + bond_check_code + '.cif'
		
				if WRITE_CIF:
					print('writing cif...')
					print()
					if len(cifname) > 255:
						cifname = cifname[0:241]+'_truncated.cif'
					write_cif(fc_placed_all, fixed_bonds, scaled_params, sc_unit_cell, outcifname, CHARGES, wrap_coords=False)

if catenation and MERGE_CATENATED_NETS:
	
	print('merging catenated cifs...')
	cat_cifs = glob.glob('output_cifs/*_CAT*.cif')

	for comb in itertools.combinations(cat_cifs, cat_count):

		builds = [name[0:-9] for name in comb]

		print(set(builds))

		if len(set(builds)) == 1:
			pass
		else:
			continue

		merge_catenated_cifs(comb, CHARGES)

	#for cif in cat_cifs:
	#	os.remove(cif)


template : fcu.cif

Number of vertices =  4
Number of edges =  24

*****************************************************************
RMSD of the compatible node BBs with assigned vertices:          
*****************************************************************

vertex V (12 connected)
     12c_Zr_1_Ch.cif deviation = 0.0 (within tolerance)
* 1 compatible building blocks out of 1 available for node V *

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
vertex assignment :  ['v1-12c_Zr_1_Ch.cif']
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
edge assignment :  ('1B_2F_Ch.cif',)
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

scaling unit cell and vertex positions...
optimizing with local minimization algorithm L-BFGS-B...

The final objective function value

In [36]:
cleaved_placed_nodes[:6]

array([['O1', '-6.435695587837782e-16', '1.9727999999999983',
        '3.529400000000001', '-0.861922', 'O1', '2', 'NODE'],
       ['O2', '9.237988984610325e-16', '1.9728000000000019',
        '-3.5294000000000003', '-0.863582', 'O2', '2', 'NODE'],
       ['O3', '1.9728000000000003', '3.5293999999999994',
        '1.4012384878051803e-15', '-0.880269', 'O3', '2', 'NODE'],
       ['O4', '-1.9727999999999999', '3.5294000000000003',
        '1.4469350732399215e-15', '-0.867827', 'O4', '2', 'NODE'],
       ['O5', '-3.5293999999999994', '1.9728000000000006',
        '8.511968719415719e-16', '-0.863731', 'O5', '2', 'NODE'],
       ['O6', '3.5294000000000003', '1.9727999999999997',
        '7.694442722693417e-16', '-0.877439', 'O6', '2', 'NODE']],
      dtype='<U32')

In [37]:
for i in [cleaved_placed_edges,placed_edges,cleaved_placed_nodes,placed_nodes]:
    print(len(i))
    
## edges=cleaved_placed_edges.reshape((len(TG.edges()),int(len(cleaved_placed_edges)/len(TG.edges())),cleaved_placed_edges.shape[1]))
## edges_x = []
## for i in range(edges.shape[0]):
##     ed = edges[i]
##     row_x =[]
##     for row in ed[0:6]:
##         if re.sub('[0-9]','',row[5]) == 'X':
##             row_x.append(row)
##             #print(row)
##             
##     edges_x.append(np.asarray(row_x))
## 
## edges_x

384
240
216
216


In [38]:
def placed_arr(placed_all):
    placed_all_arr=np.empty((len(placed_all),len(placed_all[0])),dtype=object)
    for i in range(len(placed_all)):
            line = placed_all[i]
            placed_all_arr[i]=line

    placed_all_arr[:,1:4]=placed_all_arr[:,1:4].astype(float)
    placed_all_arr[:,6]=placed_all_arr[:,6].astype(int)
    res_id=np.unique(placed_all_arr[:,6]).astype(int)
    return placed_all_arr,res_id

def fetch_node_withidx(placed_node,idx_list):
    res_id_list = [i+1 for i in idx_list ]
    res=[]
    for n in res_id_list:
        res.append(placed_node[placed_node[:,6]==n])
    return np.vstack(res)

    
def fetch_edge_withidx(placed_edge,idx_list):
    res_id_list = [-1*i-1 for i in idx_list ]
    res=[]
    for n in res_id_list:
        res.append(placed_edge[placed_edge[:,6]==n])
    return np.vstack(res)

def temp_xyz(output,placed_all):
    atoms_number = len(placed_all)
    newxyz = []
    with open( output, "w") as fp:
        newxyz.append(str(atoms_number) + "\n" + "generated by MOF_BUILD" + "\n")
        for i in range(atoms_number):

            value_label = placed_all[i][0]   # atom_label
            value_label = re.sub(r"\d", "", value_label)
            value_x = float(placed_all[i][1])  # x
            value_y = float(placed_all[i][2])   # y
            value_z = float(placed_all[i][3])   # z
            formatted_line = "%-5s%8.3f%8.3f%8.3f" % (
                value_label,
                value_x,
                value_y,
                value_z,
            )
            newxyz.append(formatted_line + "\n")
        fp.writelines(newxyz)

def fc_assign_res_idx(placed_all):
    placed_all_arr=np.empty((len(placed_all),len(placed_all[0])),dtype=object)
    for i in range(len(placed_all)):
        line = placed_all[i]
        placed_all_arr[i]=line

    placed_all_arr[:,1:4]=placed_all_arr[:,1:4].astype(float)
    placed_all_arr[:,6]=placed_all_arr[:,6].astype(int)
    res_id=list(np.unique(placed_all_arr[:,6]).astype(int))
    
    res_id.sort(reverse=True)
    res_list=[]
    for i in range(len(res_id)):
        r_id = res_id[i]
        res_arr= placed_all_arr[placed_all_arr[:,6]==r_id]
        vec = res_arr[:,1:4]
        cvec = np.dot(vec,np.linalg.inv(sc_unit_cell))
        #moded_cvec = np.mod(cvec, 1)
        #cvec=moded_cvec   
        res_arr[:,1:4] = cvec
        res_arr[:,6] = i+1
        res_list.append(res_arr)

    return np.vstack((res_list))

def outxyz(output,all_array):

    atoms_number = len(all_array)
    newxyz = []
    with open(output, "w") as fp:
        newxyz.append(str(atoms_number) + "\n" + "generated by MOF_BUILD" + "\n")
        for i in range(atoms_number):
            row = all_array[i]
            value_label = row[0]   # atom_label
            value_label = re.sub(r"\d", "", value_label)
            value_x = sc_a*float(row[1])  # x
            value_y = sc_b*float(row[2])  # y
            value_z = sc_c*float(row[3])  # z
            formatted_line = "%-5s%8.3f%8.3f%8.3f" % (
                value_label,
                value_x,
                value_y,
                value_z,
            )
            newxyz.append(formatted_line + "\n")
        fp.writelines(newxyz)

def outpdb(output,all_array):

        atoms_number = len(all_array)

        newpdb = []
        newpdb.append("generated by MOF_BUILD" + "\n")
        newpdb.append(str(atoms_number) + "\n")
        with open(output, "w") as fp:
            # Iterate over each line in the input file
            for i in range(atoms_number):
                line = all_array[i]
                # Extract values based on their positions in the format string
                value1 = "ATOM"
                value2 = int(i + 1)
                value3 = line[5]
                value4 = str(line[7])[0]  # residue
                value5 = line[6]  # residue number
                value6 = sc_a*float(line[1])  # x
                value7 = sc_b*float(line[2])  # y
                value8 = sc_c*float(line[3])  # z
                value9 = "1.00"
                value10 = "0.00"
                value11 = re.sub(r"\d", "", line[0]) # note
                # Format the values using the specified format string
                formatted_line = "%-6s%5d%5s%4s%10d%8.3f%8.3f%8.3f%6s%6s%4s" % (
                    value1,
                    value2,
                    value3,
                    value4,
                    value5,
                    value6,
                    value7,
                    value8,
                    value9,
                    value10,
                    value11,
                )
                newpdb.append(formatted_line + "\n")
            fp.writelines(newpdb)

def outgro(output,all_array):
        atoms_number = len(all_array)

        newgro = []
        with open(output, "w") as fp:
            newgro.append("generated by MOF_BUILD" + "\n" + str(atoms_number) + "\n")
            for i in range(atoms_number):
                line = all_array[i]

                value_atom_number = int(i + 1)  # atom_number
                value_label = line[0]  # atom_label
                value_resname = str(line[7])[0]  # residue_name
                value_resnumber = line[6] # residue number
                value_x = 0.1*sc_a*float(line[1])  # x
                value_y = 0.1*sc_b*float(line[2])  # y
                value_z = 0.1*sc_c*float(line[3])  # z
                formatted_line = "%5d%-5s%5s%5d%8.3f%8.3f%8.3f" % (
                    value_resnumber,
                    value_resname,
                    value_label,
                    value_atom_number,
                    value_x,
                    value_y,
                    value_z,
                )
                newgro.append(formatted_line + "\n")
            tail = "5 5 5 \n"
            newgro.append(tail)
            fp.writelines(newgro)
            
def supercell_array(supercell,array):
    s_cell_list=[]
    s_cell_list_append = s_cell_list.append
    for i in supercell:
        points = array[:,1:4]
        new_points= points+i
        new_array=np.hstack((array[:,0:1],new_points,array[:,4:]))
        s_cell_list_append(new_array)
    return np.vstack((s_cell_list))
##Frame
def Carte_points_generator(x_num, y_num, z_num):
        """this function is to generate a group of 3d points(unit=1) defined by user for further grouping points"""
        unit_dx, unit_dy, unit_dz = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])
        # add x layer
        points = np.array([0, 0, 0])
        for i in range(0, x_num + 1):
            points = np.vstack((points, i * unit_dx))
        # add y layer
        points_x = points
        for i in range(0, y_num + 1):
            points = np.vstack((points, points_x + i * unit_dy))
        # add z layer
        points_xy = points
        for i in range(0, z_num + 1):
            points = np.vstack((points, points_xy + i * unit_dz))
        points = np.unique(points, axis=0)
        return points
    
    

def tric_points(Carte_points,unit_cell):
        #Carte_points = Frame.Carte_points_generator(
        #    self.x_num,
        #    self.y_num,
        #    self.z_num,
        #)
        if len(Carte_points)>0:
            supercell_tric_points = np.round(np.dot(Carte_points, unit_cell),3)
            return supercell_tric_points
        else:
            return np.empty((0,3))
        

def cleave_supercell_boundary(supercell, supercell_boundary):
    print(
        f"supercell {supercell.shape[0]}, supercell_boundary {supercell_boundary.shape[0]}"
    )
    supercell_inside_list = [
        i for i in supercell.tolist() if i not in supercell_boundary.tolist()
    ]
    print(f"cleaved supercell(not in latter but in former) {len(supercell_inside_list)}")
    return np.array(supercell_inside_list)

class Frame():
    def __init__(self,supercell_xyz,unit_cell):
         self.supercell_xyz = supercell_xyz
         self.unit_cell = unit_cell
         
    def supercell_generator(self):
        super_cell_x, super_cell_y, super_cell_z = self.supercell_xyz
        supercell_Carte = Carte_points_generator(super_cell_x, super_cell_y, super_cell_z)
        self.supercell_Carte = supercell_Carte
        # print(f"supercell_Carte{supercell_Carte}")
        outer_supercell_Carte = Carte_points_generator(
            super_cell_x + 2, super_cell_y + 2, super_cell_z + 2
        )-np.array([1,1,1])
        # print(f"outer_Carte{outer_supercell_Carte}")

        supercell_boundary_Carte = supercell_Carte[
            (supercell_Carte[:, 0] == super_cell_x)
            | (supercell_Carte[:, 0] == 0)
            | (supercell_Carte[:, 1] == super_cell_y)
            | (supercell_Carte[:, 1] == 0)
            | (supercell_Carte[:, 2] == super_cell_z)
            | (supercell_Carte[:, 2] == 0)
        ]


        supercell_inside_Carte = cleave_supercell_boundary(supercell_Carte,supercell_boundary_Carte)
        layer_out_supercell_Carte = cleave_supercell_boundary(outer_supercell_Carte,supercell_Carte)

        boundary_Carte_x  = supercell_boundary_Carte[supercell_boundary_Carte[:,0]==super_cell_x]
        boundary_Carte_x_ = supercell_boundary_Carte[supercell_boundary_Carte[:,0]==0]
        boundary_Carte_y  = supercell_boundary_Carte[supercell_boundary_Carte[:,1]==super_cell_y]
        boundary_Carte_y_ = supercell_boundary_Carte[supercell_boundary_Carte[:,1]==0]
        boundary_Carte_z  = supercell_boundary_Carte[supercell_boundary_Carte[:,2]==super_cell_z]
        boundary_Carte_z_ = supercell_boundary_Carte[supercell_boundary_Carte[:,2]==0]

        layer_out_Carte_x  = layer_out_supercell_Carte[layer_out_supercell_Carte[:,0]>super_cell_x]
        layer_out_Carte_x_ = layer_out_supercell_Carte[layer_out_supercell_Carte[:,0]<0]
        layer_out_Carte_y  = layer_out_supercell_Carte[layer_out_supercell_Carte[:,1]>super_cell_y]
        layer_out_Carte_y_ = layer_out_supercell_Carte[layer_out_supercell_Carte[:,1]<0]
        layer_out_Carte_z  = layer_out_supercell_Carte[layer_out_supercell_Carte[:,2]>super_cell_z]
        layer_out_Carte_z_ = layer_out_supercell_Carte[layer_out_supercell_Carte[:,2]<0]

        self.supercell=tric_points(supercell_Carte,self.unit_cell)
        self.boundary_supercell     = tric_points(supercell_boundary_Carte, self.unit_cell)
        self.inside_supercell       = tric_points(supercell_inside_Carte,   self.unit_cell)
        self.layer_out_supercell    = tric_points(layer_out_supercell_Carte,self.unit_cell)
        self.boundary_supercell_x   = tric_points(boundary_Carte_x , self.unit_cell)
        self.boundary_supercell_x_  = tric_points(boundary_Carte_x_, self.unit_cell)
        self.boundary_supercell_y   = tric_points(boundary_Carte_y , self.unit_cell)
        self.boundary_supercell_y_  = tric_points(boundary_Carte_y_, self.unit_cell)
        self.boundary_supercell_z   = tric_points(boundary_Carte_z , self.unit_cell)
        self.boundary_supercell_z_  = tric_points(boundary_Carte_z_, self.unit_cell)
        self.layer_out_supercell_x  = tric_points(layer_out_Carte_x ,self.unit_cell)
        self.layer_out_supercell_x_ = tric_points(layer_out_Carte_x_,self.unit_cell)
        self.layer_out_supercell_y  = tric_points(layer_out_Carte_y ,self.unit_cell)
        self.layer_out_supercell_y_ = tric_points(layer_out_Carte_y_,self.unit_cell)
        self.layer_out_supercell_z  = tric_points(layer_out_Carte_z ,self.unit_cell)
        self.layer_out_supercell_z_ = tric_points(layer_out_Carte_z_,self.unit_cell)



In [39]:
placed_nodes_arr,nodes_id=placed_arr(cleaved_placed_nodes)
placed_edges_arr,edges_id=placed_arr(cleaved_placed_edges)

nodes_idx =np.arange(1,1+len(TG.nodes()),1).tolist()
edges_idx =np.arange(1,1+len(TG.edges()),1).tolist()
#t_nodes_idx = [4]#nodes_idx#[3,5,7,8]
#t_edges_idx = edges_idx
#target_nodes = fetch_node_withidx(placed_nodes_arr,t_nodes_idx)
#target_edges = fetch_edge_withidx(placed_edges_arr,t_edges_idx)
#target_all = np.vstack((target_nodes,target_edges))
#frame=Frame([1,1,1],sc_unit_cell)
#frame.supercell_generator()
#target_wrapped_array = fc_assign_res_idx(target_all)
#s_array = supercell_array(Carte_points_generator(1,1,1),target_wrapped_array)
#
#
#xyz='output_cifs/03te.xyz'
#pdb='output_cifs/04te.pdb'
#gro='output_cifs/07te.gro'
#gro1='output_cifs/071te.gro'
##outxyz(xyz,all_array)
#outpdb(pdb,s_array)
#outgro(gro,target_wrapped_array)
#outgro(gro1,s_array)
#temp_xyz("0term_supercell.xyz",a)

In [40]:
##skip!!!
##def _place_nodes(nvecs):

#import numpy as np
#from numpy.linalg import norm
#from bbcif_properties import bb2array, X_vecs, bbbonds, bbcharges
#from place_bbs import superimpose
#
#import itertools
#import re
#
#placed_nbb_coords = []
#placed_nbb_coords_extend = placed_nbb_coords.extend
#all_bonds = []
#all_bonds_extend = all_bonds.extend
#ind_seg = 0
#bbind = 1
#
#for n in nvecs:
#    bbind = bbind + 1
#    name,cvec,cif,nvec = n
#    ll = 0
#
#    for v in nvec:
#        mag = np.linalg.norm(v - np.average(nvec, axis = 0))
#        if mag > ll:
#            ll = mag
#
#    bbxvec = np.array(X_vecs(cif,'nodes',False))
#
#    #if ORIENTATION_DEPENDENT_NODES:
#    nbbxvec = bbxvec
#    #else:
#    #	nbbxvec = np.array([ll*(v / np.linalg.norm(v)) for v in bbxvec])
#
#    min_dist,rot,tran = superimpose(nbbxvec,nvec)
#
#    all_bb = bb2array(cif, 'nodes') # extract node cif atoms as all_bb (building block)
#    all_coords = np.array([v[1] for v in all_bb]) #node cif: atoms coordinate
#    all_inds = np.array([v[0] for v in all_bb]) #node cif: atom identifier
#    chg, elem = bbcharges(cif, 'nodes')  ##node cif: atom charge and atom element 
#    all_names = [o + re.sub('[A-Za-z]','',p) for o,p in zip(elem,all_inds)] #all_names: element + numeric part in identifier relace X to C
#    print(f"ind_seg{ind_seg}")
#    
#    '''!!!'''
#    #may change ind_seg but can use residue count for every node, each loop is a complete node 
#    all_names_indices = np.array([int(re.sub('[A-Za-z]','',e)) for e in all_names]) + ind_seg
#
#    elem_dict = dict((k,'') for k in all_inds)
#    for i,j in zip(all_inds, elem):
#        elem_dict[i] = j
#
#    ind_dict = dict((k,'') for k in all_inds)
#    for i,j in zip(all_inds, all_names_indices):
#        ind_dict[i] = j
#
#    '''!!!'''
#    #may use bonds to add dummy atoms , filter 'S' metal-Oxygen bond
#    bonds = bbbonds(cif, 'nodes')
#
#    anf = [str(elem_dict[n]) + str(ind_dict[n]) for n in all_inds]
#
#    abf = []
#    for b in bonds:
#        b1 = str(elem_dict[b[0]]) + str(ind_dict[b[0]])
#        b2 = str(elem_dict[b[1]]) + str(ind_dict[b[1]])
#        abf.append([b1,b2] + b[2:])
#    
#    '''!!!'''
#    aff_all = np.dot(all_coords,rot) + cvec #rotate and translate cif_node to correswponding node in G
#    
#    laff_all = np.c_[anf, aff_all, chg, all_inds, [bbind] * len(anf)]
#
#    placed_nbb_coords_extend(laff_all)
#    all_bonds_extend(abf)
#    ind_seg = ind_seg + len(all_names)
#
#	#return placed_nbb_coords, all_bonds

In [41]:
## def count_num(xyz,num):
##     count = 0
##     for i in np.round(xyz,2):
##         if i==num:
##             count+=1
##     return count
## 
## def boundary_sign(x):
##     if x == 0:
##         return '-'
##     if x == 1:
##         return '+'
## 
## def boundary_node_sign(xyz):
##     x,y,z = xyz
##     s=[]
##     s_append = s.append
##     if boundary_sign(x)=='+':
##         s_x = 'x+'
##         s_append(s_x)
##     if boundary_sign(x)=='-':
##         s_x = 'x-'
##         s_append(s_x)
##     if boundary_sign(y)=='+':
##         s_y = 'y+'
##         s_append(s_y)
##     if boundary_sign(y)=='-':
##         s_y = 'y-'
##         s_append(s_y)
##     if boundary_sign(z)=='+':
##         s_z = 'z+'
##         s_append(s_z)
##     if boundary_sign(z)=='-':
##         s_z = 'z-'
##         s_append(s_z)
##     return s
## 
## def check_b_face_point(point):
##     xyz= np.round(point,2)
##     #check x:
##     zero_num = count_num(xyz,0)
##     one_num = count_num(xyz,1)
##     if  zero_num+one_num == 0:
##         #print("not boundary",[],[])
##         return ("NB",'','')
## 
##     else:
##         s = boundary_node_sign(xyz)
##         #print('boundary nodes',s,xyz)
##         if len(s)==3:
##             return ('V',' '.join(s),xyz)
##         if len(s)==2:
##             return ('E',' '.join(s),xyz)
##         if len(s)==1:
##             return ('F',' '.join(s),xyz)


## nodes_name = list(TG.nodes())
## nodes_dict = TG.nodes(data=True)
## 
## #check node(center) is boundary or not 
## boundary_nodes_l=[(node,check_b_face_point(nodes_dict[node]['fcoords'])) for node in nodes_name]
## 
## bn_face_list=list(set([i[1][1]for i in boundary_nodes_l if i[1][1] != ''] ))
## bn_face_dict = {}
## for bn_f in bn_face_list:
##     bn_face_dict[bn_f] = [i[0] for i in boundary_nodes_l if i[1][1]==bn_f ]
## bn_face_dict                     
## edges=cleaved_placed_edges.reshape((len(TG.edges()),int(len(cleaved_placed_edges)/len(TG.edges())),cleaved_placed_edges.shape[1]))
## edges_x = {}
## for i in range(edges.shape[0]):
##     ed = edges[i]
##     row_x =[]
##     for row in ed[0:6]:
##         if re.sub('[0-9]','',row[5]) == 'X':
##             row_x.append(row)
##             #print(row)
##         e_idx = -1*int(row[6])-1
##     edges_x[e_idx]=(np.asarray(row_x))
## 

In [42]:
## def boundary_edge_sign(xyz):
##    x,y,z = xyz
##    s=[]
##    n_x,n_y,n_z = '','',''
##    s_append = s.append
##    if x>1:
##        s_x = 'x+'
##        n_x = '1'
##        s_append(s_x)
##        
##    if x<0:
##        s_x = 'x-'
##        n_x = '4'
##        s_append(s_x)
##    if y>1:
##        s_y = 'y+'
##        n_y = '2'
##        s_append(s_y)
##    if y<0:
##        s_y = 'y-'
##        n_y = '5'
##        s_append(s_y)
##    if z>1:
##        s_z = 'z+'
##        n_z = '3'
##        s_append(s_z)
##    if z<0:
##        s_z = 'z-'
##        n_z = '6'
##        s_append(s_z)
##
##    
##    return s,n_x+n_y+n_z
## 
## 
## 
## 
## def outercell_score(frame):
##    cell_score=[]
##    for i in range(len(frame.layer_out_supercell)):
##        score_list=[]
##        if frame.layer_out_supercell[i].tolist() in frame.layer_out_supercell_x_.tolist():
##            score_list.append('x+')
##        if frame.layer_out_supercell[i].tolist() in frame.layer_out_supercell_x.tolist():
##            score_list.append('x-')
##        if frame.layer_out_supercell[i].tolist() in frame.layer_out_supercell_y_.tolist():
##            score_list.append('y+')
##        if frame.layer_out_supercell[i].tolist() in frame.layer_out_supercell_y.tolist():
##            score_list.append('y-')
##        if frame.layer_out_supercell[i].tolist() in frame.layer_out_supercell_z_.tolist():
##            score_list.append('z+')
##        if frame.layer_out_supercell[i].tolist() in frame.layer_out_supercell_z.tolist():
##            score_list.append('z-')
##        cell_score.append(' '.join(score_list))
##    print(f"outercell_score:\n{cell_score}")
##    return cell_score
## 
## def boundarycell_score(frame):
##    cell_score=[]
##    for i in range(len(frame.boundary_supercell)):
##        score_list=[]
##        if frame.boundary_supercell[i].tolist() in frame.boundary_supercell_x.tolist():
##            score_list.append('x+')
##        if frame.boundary_supercell[i].tolist() in frame.boundary_supercell_x_.tolist():
##            score_list.append('x-')
##        if frame.boundary_supercell[i].tolist() in frame.boundary_supercell_y.tolist():
##            score_list.append('y+')
##        if frame.boundary_supercell[i].tolist() in frame.boundary_supercell_y_.tolist():
##            score_list.append('y-')
##        if frame.boundary_supercell[i].tolist() in frame.boundary_supercell_z.tolist():
##            score_list.append('z+')
##        if frame.boundary_supercell[i].tolist() in frame.boundary_supercell_z_.tolist():
##            score_list.append('z-')
##        cell_score.append(' '.join(score_list))
##    print(f"boundary_cell_score:\n{cell_score}")
##    return cell_score
## 
## def reverse_face_number(abc):
##    slice=[]
##    for i in abc.split():
##        slice.append(i)
##    return slice
## 
## def check_listA_in_listB_any(listA, listB):
##    return any(item in listB for item in listA)
## 
## def check_listA_in_listB_all(listA, listB):
##    return all(item in listB for item in listA)
## 
## def find_eachcell_boundary_linkers(frame,multifaces_linker_dict):
##    multifaces_linker_dict_key = list(multifaces_linker_dict) if multifaces_linker_dict else []
##    cell_score = boundarycell_score(frame)
##    b_supercell_linker_indices=[]
##    b_supercell_term_indices=[]
##    for i in range(len(frame.boundary_supercell)):
##        b_linkers_index=[]
##        b_terms_index=[]
##        for face_number in  multifaces_linker_dict_key:
##            face = reverse_face_number(face_number)
##            if check_listA_in_listB_any(face,cell_score[i]) is not True: #not this boundary,then add this boundary linker
##                b_linkers_index += multifaces_linker_dict[face_number]        
##            else: #if this boundary,then add this boundary termination
##                b_terms_index += multifaces_linker_dict[face_number]   
##        print(
##            f"supercell:{i}\n\t"
##            f"b_terms_index:{b_terms_index}\n\t"
##            f"b_linkers_index:{b_linkers_index}\n"
##        )
##        b_supercell_linker_indices.append(b_linkers_index)
##        b_supercell_term_indices.append(b_terms_index)
##
##    return b_supercell_linker_indices,b_supercell_term_indices
## 
## 
## def find_envelope_boundary_terms(frame,multifaces_linker_dict):
##    multifaces_linker_dict_key = list(multifaces_linker_dict) if multifaces_linker_dict else []
##    cell_score = outercell_score(frame)
##
##    e_supercell_term_indices=[]
##    for i in range(len(frame.layer_out_supercell)):
##        e_terms_index=[]
##        for face_number in  multifaces_linker_dict_key:
##            face = reverse_face_number(face_number)
##            #print(f"face_number,face,cell_score[i]{face_number,face,cell_score[i]}")
##            #if len(cell_score[i])>1: # at edge or vertex
##            #    if face==cell_score[i]: #this outer boundary,then add term
##            #        e_terms_index += multifaces_linker_dict[face_number]    
##            #else:
##            if check_listA_in_listB_all(face,cell_score[i]) is True: # on the face 
##                    e_terms_index += multifaces_linker_dict[face_number]    
##
##        print(
##            f"envelope_cell:{i}\n\t"
##            f"e_terms_index:{e_terms_index}"
##        )
##        e_supercell_term_indices.append(e_terms_index)
##
##    return e_supercell_term_indices
## 
## 
## 
## bl_linker_face_list=[]
## bl_face_list=[]
## for i in range(len(edges_x)):
##     ex_c=edges_x[i+1][:,1:4].astype(float)
##     ex_f=np.round(np.dot(ex_c,np.linalg.inv(sc_unit_cell)),3)
##     if np.any(ex_f<0) or np.any(ex_f>1):
##         s0,n_0 = boundary_edge_sign(ex_f[0])
##         s1,n_0 = boundary_edge_sign(ex_f[1])
##         if s0==s1:
##             bl_linker_face_list.append((i+1,' '.join(s0)))
##             bl_face_list.append(' '.join(s0))
## 
## bl_face_dict = {}
## bl_face_list = set(bl_face_list)
## for bl_f in bl_face_list:
##     bl_face_dict[bl_f] = [i[0] for i in bl_linker_face_list if i[1]==bl_f]
## bl_face_dict
## 
## bl_idx=[i[0] for i in bl_linker_face_list]
## il_idx=[idx for idx in edges_idx if idx not in bl_idx]
## bl_idx,il_idx
## bn_idx = [int(re.sub(r'\D','',bn_face_dict[i][0])) for i in bn_face_dict]
## in_idx = [i for i in nodes_idx if i not in bn_idx]
## bn_idx,in_idx
## 
## 
## 
## boundary_cell_linker_idx,boundary_cell_terms_idx=find_eachcell_boundary_linkers(frame,bl_face_dict)
## e_supercell_term_idx=find_envelope_boundary_terms(frame,bl_face_dict)
## 
## target_nodes = fetch_node_withidx(placed_nodes_arr,t_nodes_idx)
## target_edges = fetch_edge_withidx(placed_edges_arr,t_edges_idx)
## 
## 
## ##supercell_inside.shape,supercell_boundary.shape,supercell.shape
## #### supercell
## ##   1st: supercell_inside_arr
## ##   2nd: supercell_boundary
## #
## inside_supercell_linker_list=[]
## inside_supercell_linker_info_list=[]
## if len(frame.inside_supercell)>0:
##     for i in range(len(frame.inside_supercell)):
##         inside_cell_linker=frame.inside_supercell[i]+ placed_nodes_arr[1:4]#= array_3Dto2D(self.cell_linkers_array)
##         inside_cell_linker_info=placed_nodes_arr[:,0:1] #= array_3Dto2D(self.cell_linkers_info_array)
##         inside_supercell_linker_list.append(inside_cell_linker)
##         inside_supercell_linker_info_list.append(inside_cell_linker_info)
## #vstack all linkers and info 
## boundary_linker_term_list=[]
## boundary_linker_term_info_list=[]
## for i in range(len(frame.boundary_supercell)):
## #    #incell linkers which are fixed number and position
##     incell_linkers_arr = fetch_edge_withidx(placed_edges_arr,il_idx)
##     fixed_incell_linkers = frame.boundary_supercell[i]+incell_linkers_arr[:,1:4]
##     fixed_incell_linkers_info = incell_linkers_arr[:,0:1]
## #    # unboundary linkers and boundary terms 
##     if (len(boundary_cell_linker_idx[i])+len(boundary_cell_terms_idx[i]))>0:
##         inner_linker_X = None
##         inner_linker_X = frame.boundary_supercell[i]+fetch_edge_withidx(placed_edges_arr,boundary_cell_linker_idx[i])[:,1:4]
##         #print(inner_linker_X.shape,boundary_cell_linker_indices[i])
##         inner_linker_info_X = fetch_edge_withidx(placed_edges_arr,boundary_cell_linker_idx[i])[:,0:1]
##         #term_Oxy4_X =  frame.boundary_supercell[i]+array_3Dto2D(term_boundary_linker_Oxy4[boundary_cell_terms_indices[i]])
##         #term_Oxy4_info_X = array_3Dto2D(term_boundary_linker_info_Oxy4[boundary_cell_terms_indices[i]])
##         term_Oxy4_X =  frame.boundary_supercell[i]+fetch_edge_withidx(placed_edges_arr,boundary_cell_terms_idx[i])[:,1:4]
##         term_Oxy4_info_X = fetch_edge_withidx(placed_edges_arr,boundary_cell_terms_idx[i])[:,0:1]
## #
##         boundary_linker_term_list.append(inner_linker_X)
##         boundary_linker_term_list.append(term_Oxy4_X)
## #    
##         boundary_linker_term_info_list.append(inner_linker_info_X)
##         boundary_linker_term_info_list.append(term_Oxy4_info_X)
## # 
##         #print(f"{i}\ninner_linker_X{inner_linker_X.shape},term_Oxy4_X{term_Oxy4_X.shape}")
## #
##     boundary_linker_term_list.append(fixed_incell_linkers)
##     boundary_linker_term_info_list.append(fixed_incell_linkers_info)
## #    
## e_term_Oxy4_list=[]
## e_term_Oxy4_info_list=[]
## ##
## for i in range(len(frame.layer_out_supercell)):
##     if len(e_supercell_term_idx[i])>0:
## ##    #e_term_Oxy4_X =  frame.layer_out_supercell[i]+array_3Dto2D(cell_boundary_linkers[e_supercell_term_indices[i]])
## ##    #e_term_Oxy4_info_X = array_3Dto2D(cell_boundary_linkers_info[e_supercell_term_indices[i]])
##         e_term_Oxy4_X =  frame.layer_out_supercell[i]+fetch_edge_withidx(placed_edges_arr,e_supercell_term_idx[i])[:,1:4]
##         e_term_Oxy4_info_X = fetch_edge_withidx(placed_edges_arr,e_supercell_term_idx[i])[:,0:1]
##         e_term_Oxy4_list.append(e_term_Oxy4_X)
##         e_term_Oxy4_info_list.append(e_term_Oxy4_info_X)
## 
##     
## 
## 
## 
## def array_3Dto2D(array3d):
##     if len(array3d.shape) == 3:
##         array2d = array3d.reshape(
##             (array3d.shape[0] * array3d.shape[1], array3d.shape[2])
##         )
##         return array2d
##     elif len(array3d.shape) == 4:  # only happen with one node
##         array2d = array3d.reshape(
##             (array3d.shape[0] * array3d.shape[2], array3d.shape[3])
##         )
##         return array2d
##     else:
##         print(f"array shape is {array3d.shape} not 3D array")
##         return array3d
## 
## def list_2_2darray(list):
##     #print(len(list))
##     if len(list)>0:
##         array3d = np.array(list)
##         return array_3Dto2D(array3d)
##     else:
##         return np.empty((0,3))
## def list_2_2dinfo(list):
##     #print(len(list))
##     if len(list)>0:
##         array3d = np.array(list)
##         return array_3Dto2D(array3d)
##     else:
##         return np.empty((0,1))
##     
## def savexyz(file_name):
##     with open(file_name,'r') as f:
##         lines = f.readlines()
## 
##     title = []
##     title.append(str(len(lines))+'\n')
##     title.append('generated by MOF_auto\n')
##     newxyz=title+lines
## 
##     with open(file_name,'w') as fp:
##         fp.writelines(newxyz)
##     print(f"generated xyz file:\t{file_name} \natoms number:\t\t {title[0]}")
## 
## #inside_supercell_linker_list,inside_supercell_linker_info_list
## #boundary_linker_term_list,boundary_linker_term_info_list
## 
## super_node = []
## super_node_info =[]
## 
## for i in range(len(frame.supercell)):
##     super_node.append(frame.supercell[i]+placed_nodes_arr[:,1:4])
##     super_node_info.append(placed_nodes_arr[:,0:1])
## 
## super_node_array2d=list_2_2darray(super_node)
## super_node_info2d=list_2_2dinfo(super_node_info)
## 
## super_node_all = np.hstack((super_node_info2d,super_node_array2d))
## 
## supercell_linker_term=np.vstack((inside_supercell_linker_list+boundary_linker_term_list+e_term_Oxy4_list))
## supercell_linker_term_info=np.vstack((inside_supercell_linker_info_list+boundary_linker_term_info_list+e_term_Oxy4_info_list))
## 
## supercell_linkers_terms_xyz = np.hstack((supercell_linker_term_info,supercell_linker_term))
## 
## supercell_linkers_terms_xyz = supercell_linkers_terms_xyz.astype(object)
## supercell_linkers_terms_xyz[:, 0] = supercell_linkers_terms_xyz[:, 0].astype(str)
## supercell_linkers_terms_xyz[:, 1:] = supercell_linkers_terms_xyz[:, 1:].astype(float)
## 
## a=np.vstack((super_node_all,supercell_linkers_terms_xyz))
## 
## print(f"all atoms num:\t{len(a)}")
## #np.savetxt("0term_supercell.xyz", a, delimiter=" ", fmt="%5s%8.3f%8.3f%8.3f")
## #savexyz("0term_supercell.xyz")

In [43]:

#target_wrapped_array = fc_assign_res_idx(target_all)
#s_array = supercell_array(Carte_points_generator(0,0,1),target_wrapped_array)


#xyz='output_cifs/03te.xyz'
#pdb='output_cifs/04te.pdb'
#gro='output_cifs/07te.gro'
gro1='output_cifs/011te.gro'
xyz1='output_cifs/011te.xyz'
#outxyz(xyz,all_array)
#outpdb(pdb,s_array)
#outgro(gro,target_wrapped_array)
#outxyz(xyz1,s_array)
#outgro(gro1,s_array)




frame=Frame(supercell,sc_unit_cell)
frame.supercell_generator()

t_nodes_idx = nodes_idx#[3,5,7,8]
t_edges_idx = edges_idx
target_nodes = fetch_node_withidx(placed_nodes_arr,t_nodes_idx)
target_edges = fetch_edge_withidx(placed_edges_arr,t_edges_idx)
target_all = np.vstack((target_nodes,target_edges))

supercell 8, supercell_boundary 8
cleaved supercell(not in latter but in former) 0
supercell 64, supercell_boundary 8
cleaved supercell(not in latter but in former) 56


In [44]:
already=[]
rescount = []
already_append=already.append
rescount_append = rescount.append
for i in target_all[:,-2]:
    if i not in already:
        already_append(i)
    if i in already:
        rescount_append(len(already))

fvec_all_info = np.hstack((target_all[:,0:1],target_all[:,4:]))
info = fvec_all_info
super_ne = []
super_ne_info =[]
fvec_all = np.dot(target_all[:,1:4],np.linalg.inv(sc_unit_cell))
for i in range(len(frame.supercell)):
    super_ne.append(frame.supercell_Carte[i]+fvec_all) 
    info=np.concatenate((fvec_all_info, np.asarray([rescount]).T+i*len(already)), axis=1)
    super_ne_info.append(info)

super_ne_array2d=np.vstack((super_ne))
super_ne_info2d=np.vstack((super_ne_info))
moded_super_ne_array2d = np.mod(super_ne_array2d,box_bound)


In [45]:
#moded_super_node_array2d=np.dot(moded_super_node_array2d,sc_unit_cell)

#super_node_all = np.hstack((super_node_info2d,moded_super_node_array2d))

In [46]:
def split_diffs(diffs):
    diff_ele=[]
    diff_ele_append = diff_ele.append
    dx=np.array([1,0,0])
    dy=np.array([0,1,0])
    dz=np.array([0,0,1])

    for i in diffs:
        x,y,z = i
        if x !=0:
            diffx= (x*dx).tolist()
            if diffx not in diff_ele:
                diff_ele_append(diffx)
        if y !=0:
            diffy= (y*dy).tolist()
            if diffy not in diff_ele:
                diff_ele_append(diffy)
        if z !=0:
            diffz= (z*dz).tolist()
            if diffz not in diff_ele:
                diff_ele_append(diffz)
        if x*y !=0:
            diffxy = (x*dx+y*dy).tolist()
            if diffxy not in diff_ele:
                diff_ele_append(diffxy)
        if x*z !=0:
            diffxz = (x*dx+z*dz).tolist()
            if diffxz not in diff_ele:
                diff_ele_append(diffxz)
        if y*z !=0:
            diffyz = (y*dy+z*dz).tolist()
            if diffyz not in diff_ele:
                diff_ele_append(diffyz)
        if x*y*z !=0:
            diffxyz = (x*dx+y*dy+z*dz).tolist()
            if diffxyz not in diff_ele:
                diff_ele_append(diffxyz)
    return diff_ele

def check_edge_center_inbox(center,box_bound):
    a,b,c = box_bound
    x,y,z = center
    return (0<=x<=a and 0<=y<=b and 0<=z<=c)

def check_edgex_sits_inboundary(pointx1,pointx2,box_bound):
    a,b,c = box_bound
    pointx1 = pointx1.astype(float)
    pointx1 = np.round(pointx1,2)
    pointx2 = pointx2.astype(float)
    pointx2 = np.round(pointx2,2)
    doublecheck = check_edge_center_inbox(pointx1,box_bound) and check_edge_center_inbox(pointx2,box_bound) 
    return doublecheck

    

def check_nodex_inbox(nodex,box_bound):
    a,b,c = box_bound
    x,y,z = nodex
    return (0<x<=a and 0<y<=b and 0<z<=c)

def diff_rows_count_two_array(array0,array1):
    array0 = array0.astype(float)
    array1 = array1.astype(float)
    arr0= np.round(array0,3)
    arr1= np.round(array1,3)
    list0=[i for i in arr0]
    list1=[i for i in arr1]
    diff_rows=[i for i in range(len(list0)) if list0[i].tolist()!=list1[i].tolist()]
    return diff_rows
    


In [47]:
row_diff = np.any(super_ne_array2d != moded_super_ne_array2d, axis=1)
row_diff_idx=[i for i in range(len(row_diff)) if row_diff[i]]
s_fvec_all= np.hstack((super_ne_info2d,super_ne_array2d))
res_count_idx = set(s_fvec_all[:,-4])
differ_res_idx = set([s_fvec_all[i][-4] for i in row_diff_idx])
#print(differ_res_idx)


inside_res=[]
extra_res=[]
kick_res = []
inside_res_append = inside_res.append
extra_res_append = extra_res.append
kick_res_append = kick_res.append
boundary_node_res = []
boundary_node_res_append = boundary_node_res.append
for i in res_count_idx:
    if i in differ_res_idx:
        res=s_fvec_all[s_fvec_all[:,5]==i]
        original_fvec = res[:,-3:]
        original_fvec = original_fvec.astype(float)
        moded_fvec = np.mod(original_fvec,box_bound)
        row_diff = diff_rows_count_two_array(original_fvec,moded_fvec)
        if res[0,4]=='EDGE' and len(row_diff)>(res.shape[0]-3):
            #check if twoX sit on boundary with a tiny shift range
            if not check_edgex_sits_inboundary(original_fvec[2],original_fvec[5],box_bound):
                #print(f"69 {len(row_diff)}len(row_diff){res.shape[0]}original_fvec{original_fvec}\n{moded_fvec}")
                kick_res_append(i)  
        diff = [moded_fvec[i]-original_fvec[i] for i in row_diff]
        diff =np.vstack(diff).astype(float)
        diffs=np.unique(diff,axis=0)
        diff_ele=split_diffs(diffs)
        #print('\n',diff_ele,'\n',diffs,'\n')
        for diff_e in diff_ele:
            if res[0,4]=='NODE':
                t_res =  np.hstack((res[:,:6],res[:,-3:] + np.asarray(diff_e)))
                boundary_node_res_append(i)
                extra_res_append(t_res)
            elif res[0,4]=='EDGE':
                check1 = np.mean(res[[2,5],-3:],axis=0) + np.asarray(diff_e)
                check1=check1.astype(float)
                check1 = np.round(check1,4)
                #print(i,diff_e,check1)
                if check_edge_center_inbox(check1,box_bound):
                    #print(i,diff_e)
                    #print(i,np.round(t_res_xyz[:6],5),np.round(np.mod(t_res_xyz[:6],[1,1,1]),5),diff_e)
                    t_res =  np.hstack((res[:,:6],res[:,-3:] + np.asarray(diff_e)))
                    extra_res_append(t_res)
    else:
        res=s_fvec_all[s_fvec_all[:,5]==i]
        inside_res_append(res)


In [48]:
len(kick_res),kick_res,len(extra_res)

(66,
 [6,
  7,
  8,
  9,
  11,
  12,
  14,
  15,
  16,
  17,
  20,
  23,
  24,
  34,
  35,
  37,
  40,
  42,
  43,
  44,
  45,
  48,
  51,
  52,
  63,
  64,
  65,
  67,
  68,
  71,
  72,
  73,
  76,
  79,
  80,
  81,
  84,
  93,
  96,
  99,
  100,
  101,
  104,
  107,
  108,
  109,
  112,
  118,
  119,
  120,
  123,
  124,
  126,
  128,
  146,
  147,
  154,
  156,
  175,
  176,
  179,
  180,
  193,
  196,
  221,
  224],
 145)

In [49]:
extra_res_arr=np.vstack(extra_res)     
inside_res_arr = np.vstack(inside_res)

def temp_xyz(output,placed_all):
    atoms_number = len(placed_all)
    newxyz = []
    with open( output, "w") as fp:
        newxyz.append(str(atoms_number) + "\n" + "generated by MOF_BUILD" + "\n")
        for i in range(atoms_number):

            value_label = placed_all[i][0]   # atom_label
            value_label = re.sub(r"\d", "", value_label)
            value_x = float(placed_all[i][-3])  # x
            value_y = float(placed_all[i][-2])   # y
            value_z = float(placed_all[i][-1])   # z
            formatted_line = "%-5s%8.3f%8.3f%8.3f" % (
                value_label,
                value_x,
                value_y,
                value_z,
            )
            newxyz.append(formatted_line + "\n")
        fp.writelines(newxyz)
        
s_fvec_all[:,-3:] = np.dot(s_fvec_all[:,-3:],sc_unit_cell)
extra_res_arr[:,-3:] = np.dot(extra_res_arr[:,-3:],sc_unit_cell)
inside_res_arr[:,-3:] = np.dot(inside_res_arr[:,-3:],sc_unit_cell)

a1=np.vstack((inside_res_arr,extra_res_arr))
a2=np.vstack((s_fvec_all,extra_res_arr))



In [50]:
safe_res=[s for s in s_fvec_all if s[5] not in kick_res]
bare_nodeedge_array=np.vstack((safe_res,extra_res_arr))

def filt_edgex_cvec(array):
    edgex_cvec=[i for i in array if i[4]=='EDGE' and re.sub('[0-9]','',i[2]) == 'X']
    return edgex_cvec

def check_overlapX(edgex_cvec,nodex_cvec):
    dist_arr=edgex_cvec-nodex_cvec
    for i in dist_arr:
        if np.linalg.norm(i) <0.01:
            return True
    return False

def exposed_Xs_Os_boundary_node(boundary_node_res,bare_nodeedge_array):
    '''look for two nearest Oxys for every exposed(unsaturated X) in boundary nodes'''
    ex_node_cxo=[]
    ex_node_cxo_append=ex_node_cxo.append
    edgex_cvec = filt_edgex_cvec(bare_nodeedge_array)

    for i in list(set(boundary_node_res)):
        count=len(np.where(np.asarray(boundary_node_res)==i)[0])+1 # =1 is because original node + moded_nodes
        ress=bare_nodeedge_array[bare_nodeedge_array[:,5]==i]
        res_s=ress.reshape((count,int(ress.shape[0]/count),ress.shape[1]))
        for n in range(count):
            node=res_s[n]
            node_center = np.mean(node[:,-3:],axis=0)
            Xs = [k[-3:] for k in node if re.sub(r'\d','',k[2]) == 'X']
            Os = [g[-3:] for g in node if re.sub(r'\d','',g[2]) == 'O']
            Xs_fc = np.dot(Xs,np.linalg.inv(sc_unit_cell))
            Xs_fc = Xs_fc.astype(float)
            exposed_Xs_fc=[x for x in Xs_fc if not check_nodex_inbox(x.round(4),box_bound)]
            if len(exposed_Xs_fc)>0:
                exposed_Xs_cc=np.dot(exposed_Xs_fc,sc_unit_cell) 
                for x in exposed_Xs_cc:
                    if check_overlapX(edgex_cvec,x):
                        continue
                    else:
                        cdist_xos = []
                        cdist_xos_sort = []
                        cdist_xos_append=cdist_xos.append
                        cdist_xos_sort_append=cdist_xos_sort.append
                        for j in range(len(Os)):
                            cvec_o= Os[j]
                            cvec_xo = np.asarray(cvec_o)-np.asarray(x) 	
                            cdist_xo = np.linalg.norm(cvec_xo)
                            cdist_xos_append(cdist_xo)
                            cdist_xos_sort_append(cdist_xo)
                        cdist_xos_sort.sort()
                        cdist_xos_sort3rd=cdist_xos_sort[2]
                        node_ovecs_idx=[index for index,value in enumerate(cdist_xos) if value < cdist_xos_sort3rd]
                        node_ovecs=[Os[o] for o in node_ovecs_idx]
                        ex_node_cxo_append((node_center,len(exposed_Xs_fc),'exposed_X',x,'node_Opair',node_ovecs))
        
                    #print(f"center{node_center},Xs{len(exposed_Xs_fc)},'\n'{exposed_Xs_cc}")
            #print(res_s.shape)
    return ex_node_cxo

In [51]:
def filt_edgex_cvec(array):
    edgex_cvec=[i[-3:] for i in array if i[4]=='EDGE' and re.sub('[0-9]','',i[2]) == 'X']
    return edgex_cvec

def check_overlapX(edgex_cvec,nodex_cvec):
    dist_arr=edgex_cvec-nodex_cvec
    for i in dist_arr:
        if np.linalg.norm(i) <0.01:
            return True
    return False



In [52]:
def termpdb(filename):
        inputfile = str(filename)
        with open(inputfile, "r") as fp:
            content = fp.readlines()
            linesnumber = len(content)
        data = []
        for line in content:
            line = line.strip()
            if len(line)>0: #skip blank line
                if line[0:6] == "ATOM" or line[0:6] == "HETATM":
                    value_atom = line[12:16].strip()  # atom_label
                    #resname
                    #value2 = 'MOL'  # res_name

                    value_x = float(line[30:38])  # x
                    value_y = float(line[38:46])  # y
                    value_z = float(line[46:54])  # z
                    value_charge = float(line[61:66]) 
                    value_note = line[67:80].strip() # atom_note
                    #resnumber
                    try:
                        value_res_num = int(line[22:26])
                    except ValueError:
                        value_res_num = 1 
                    data.append([value_atom,value_x,value_y,value_z,value_charge,value_note,value_res_num,'TERM'])
        return data

def Xpdb(filename,X):
        data=termpdb(filename)
        X_term=[s for s in data if X in s[5]]
        return X_term

def convert_to_tuple(array):
    return (tuple(np.round(array[0],3).flatten()), tuple(np.round(array[1],3).flatten()),tuple(np.round(array[2],3).flatten()))

def is_list_A_in_B(A, B):
    # Convert both A and B into sets of tuple representations
    A_tuples = set(convert_to_tuple(a) for a in A)
    B_tuples = set(convert_to_tuple(b) for b in B)
    # Check if all elements in A are in B
    return A_tuples.issubset(B_tuples)

In [53]:
from place_bbs import superimpose

ex_node_cxo = exposed_Xs_Os_boundary_node(boundary_node_res,bare_nodeedge_array)

terms=[]
node_oovecs_record=[]
terms_append = terms.append
node_oovecs_record_append = node_oovecs_record.append

term_data=termpdb('methyl.pdb')
term_info = [i[0:1]+i[4:] for i in term_data]
term_coords = [s[1:4] for s in termpdb('methyl.pdb')]
xterm=Xpdb('methyl.pdb','X')
oterm=Xpdb('methyl.pdb','Y')
term_xvecs= [np.asarray([l[1],l[2],l[3]]) for l in xterm]
term_ovecs= [np.asarray([l[1],l[2],l[3]]) for l in oterm]
term_ovecs_c = np.mean(np.asarray(term_ovecs),axis=0)
term_coords = [s[1:4] for s in termpdb('methyl.pdb')]
term_xoovecs =term_xvecs+term_ovecs

for ex in range(len(ex_node_cxo)):
    node_x = ex_node_cxo[ex][3]
    node_opair=ex_node_cxo[ex][5]
    node_opair_c = np.mean(np.asarray(node_opair),axis=0)
    node_xoo_vecs =[(node_x-node_opair_c).astype(float)]+ [(i-node_opair_c).astype(float) for i in node_opair]
    indices = [index for index, value in enumerate(node_oovecs_record) if is_list_A_in_B(node_xoo_vecs,value[0])]
    if len(indices)==1: 
        #find index of node_oo_vecs in record 
        #print(f"found one,{indices}")
        rot = node_oovecs_record[indices[0]][1]
    else:
        _,rot,_, = superimpose(term_xoovecs-term_ovecs_c,node_xoo_vecs)
        
        node_oovecs_record_append((node_xoo_vecs,rot))
    adjusted_term_vecs = np.dot(term_coords-term_ovecs_c,rot) + node_opair_c
    adjusted_term = np.hstack((np.asarray(term_info),np.full((len(term_info),1), ex),adjusted_term_vecs))
    terms_append(adjusted_term)


In [54]:
bare_nodeedge_array.shape

(7242, 9)

In [55]:
terms_array = np.vstack((terms))

a3 = np.vstack((bare_nodeedge_array,terms_array))
temp_xyz("6term_supercell.xyz",a3)

In [56]:
a3[738:746]

array([['C68', '1.098413', 'X14', -19, 'EDGE', 50, 3.1794000000000016,
        7.3678828480468965, 31.64184854414069],
       ['O134', '-0.862004', 'O26', -19, 'EDGE', 50, 7.017882848046892,
        1.9727999999999977, 31.641848544140693],
       ['O136', '-0.867273', 'O28', -19, 'EDGE', 50, 8.574482848046891,
        3.529399999999996, 31.641848544140693],
       ['C157', '1.06982', 'X49', -19, 'EDGE', 50, 7.367882848046892,
        3.1793999999999913, 31.641848544140693],
       ['C387', '-0.210094', 'C1', -19, 'EDGE', 50, 5.615825936888262,
        6.605987072171523, 31.85610351611648],
       ['C388', '-0.090513', 'X2', -19, 'EDGE', 50, 4.268344443027285,
        6.278938405019609, 31.64184854414069],
       ['C389', '0.298055', 'C3', -19, 'EDGE', 50, 3.955344835741297,
        4.926364960593065, 31.428841588219218],
       ['C390', '-0.210097', 'C4', -19, 'EDGE', 50, 4.931462872855059,
        3.941275118572129, 31.427792413141567]], dtype=object)

In [57]:
a3.shape

(9174, 9)

In [58]:
a4=np.vstack((safe_res,extra_res_arr))
temp_xyz("5term_supercell.xyz",a4)

In [59]:
a1.shape,a2.shape,s_fvec_all.shape,a3.shape,a4.shape

((5784, 9), (8298, 9), (4800, 9), (9174, 9), (7242, 9))

In [60]:
def tempgro(output,all_array):
        atoms_number = len(all_array)

        newgro = []
        with open(output, "w") as fp:
            newgro.append("generated by MOF_BUILD" + "\n" + str(atoms_number) + "\n")
            for i in range(atoms_number):
                line = all_array[i]

                value_atom_number = int(i + 1)  # atom_number
                value_label = line[0]  # atom_label
                value_resname = str(line[4])[0]  # residue_name
                value_resnumber = line[5] # residue number
                value_x = 0.1*float(line[6])  # x
                value_y = 0.1*float(line[7])  # y
                value_z = 0.1*float(line[8])  # z
                formatted_line = "%5d%-5s%5s%5d%8.3f%8.3f%8.3f" % (
                    value_resnumber,
                    value_resname,
                    value_label,
                    value_atom_number,
                    value_x,
                    value_y,
                    value_z,
                )
                newgro.append(formatted_line + "\n")
            tail = "5 5 5 \n"
            newgro.append(tail)
            fp.writelines(newgro)

In [61]:
tempgro('1te.gro',a4)

In [62]:
#tempgro('2te.gro',safe_res)